In [1]:
import os
import sys
import math
import random
import joblib
import datetime
import numpy as np
import pandas as pd
from tqdm import tqdm
import warnings
warnings.filterwarnings(action='ignore')

In [2]:
sale = pd.read_excel(os.path.join('..', '..', '0.Data', '01_제공데이터', '2020 빅콘테스트 데이터분석분야-챔피언리그_2019년 실적데이터_v1_200818.xlsx'), skiprows = 1)

In [3]:
dust_2019 = pd.read_csv(os.path.join('..', '..', '0.Data', '03_외부데이터', '2019_dust.csv'), encoding = 'cp949')
dust_2020 = pd.read_csv(os.path.join('..', '..', '0.Data', '03_외부데이터', '2020_dust.csv'), encoding = 'cp949')
df_dust = pd.concat([dust_2019, dust_2020], axis = 0)

In [8]:
df_dust['지역'].apply(lambda x : x[:2]).unique()

array(['서울', '경기', '강원', '부산', '경남', '울산', '광주', '전북', '전남', '제주', '대구',
       '경북', '대전', '충북', '충남', '세종', '인천'], dtype=object)

In [9]:
df_dust.loc[df_dust['지역'].str.contains('경기'), '지역'].unique()

array(['경기 수원시', '경기 성남시', '경기 의정부시', '경기 안양시', '경기 광명시', '경기 안산시',
       '경기 과천시', '경기 구리시', '경기 의왕시', '경기 시흥시', '경기 남양주시', '경기 평택시',
       '경기 파주시', '경기 고양시', '경기 광주시', '경기 용인시', '경기 이천시', '경기 포천시',
       '경기 김포시', '경기 군포시', '경기 오산시', '경기 하남시', '경기 화성시', '경기 양주시',
       '경기 동두천시', '경기 안성시', '경기 여주시', '경기 연천군', '경기 가평군', '경기 양평군',
       '경기 부천시'], dtype=object)

In [10]:
df_dust.head()

,지역,망,측정소코드,측정소명,측정일시,SO2,CO,O3,NO2,PM10,PM25,주소
0,서울 중구,도시대기,111121,중구,2019010101,0.003,0.7,0.003,0.054,39.0,24.0,서울 중구 덕수궁길 15
1,서울 중구,도시대기,111121,중구,2019010102,0.003,0.8,0.002,0.056,38.0,27.0,서울 중구 덕수궁길 15
2,서울 중구,도시대기,111121,중구,2019010103,0.003,0.9,0.002,0.057,42.0,28.0,서울 중구 덕수궁길 15
3,서울 중구,도시대기,111121,중구,2019010104,0.003,0.8,0.002,0.054,42.0,31.0,서울 중구 덕수궁길 15
4,서울 중구,도시대기,111121,중구,2019010105,0.003,0.8,0.002,0.048,49.0,33.0,서울 중구 덕수궁길 15


In [16]:
df = preprocessing_dust(df_dust)

In [19]:
df.head()

,최고PM10_경기,최고PM10_광주,최고PM10_대구,최고PM10_대전,최고PM10_부산,최고PM10_서울,최고PM10_울산,최고PM10_인천,최고PM25_경기,최고PM25_광주,...,평균PM25_대구,평균PM25_대전,평균PM25_부산,평균PM25_서울,평균PM25_울산,평균PM25_인천,연도,월,일,시간
0,84.0,37.0,56.0,81.0,42.0,57.0,41.0,66.0,54.0,26.0,...,26.533333,40.333333,19.166667,25.975,15.941176,28.227273,2019,1,1,1
1,80.0,44.0,48.0,85.0,36.0,60.0,43.0,64.0,49.0,29.0,...,25.800000,41.363636,18.695652,27.200,18.411765,28.041667,2019,1,1,2
2,74.0,41.0,46.0,89.0,36.0,61.0,37.0,58.0,51.0,27.0,...,22.466667,42.272727,18.666667,28.425,20.470588,27.375000,2019,1,1,3
3,79.0,42.0,42.0,84.0,38.0,54.0,45.0,60.0,50.0,25.0,...,22.933333,41.636364,20.625000,28.000,21.647059,28.083333,2019,1,1,4
4,76.0,41.0,41.0,75.0,44.0,52.0,48.0,60.0,54.0,28.0,...,24.000000,41.363636,23.166667,28.975,20.647059,27.208333,2019,1,1,5


In [15]:
import gc

def prepColumns(df):
    df.columns = list(map(lambda x : '_'.join(x), df.columns))
    return df

def preprocessing_dust(df):
    temp = df.loc[df['지역'].str.contains('서울|경기|인천|부산|울산|대구|대전|광주')]
    del df
    gc.collect()
    value = temp['지역'].apply(lambda x : x[:2]).values
    df = temp.copy()
    df['지역'] = value
    del temp
    gc.collect()

    prep_df = df.groupby(['지역', '측정일시']).agg({
        'PM10' : [('최고PM10', np.max), ('최저PM10', np.min), ('평균PM10', np.mean)],
        'PM25' : [('최고PM25', np.max), ('최저PM25', np.min), ('평균PM25', np.mean)]
    }).reset_index()
    
    prep_df.columns = prep_df.columns = ['지역', '측정일시', '최고PM10', '최저PM10', '평균PM10', '최고PM25', '최저PM25', '평균PM25']
    
    prep_df['측정일시'] = prep_df['측정일시'].astype(str).apply(lambda x : x[:8] + '00' if x[8:] == '24' else x)
    prep_df['측정일시'] = pd.to_datetime(prep_df['측정일시'].astype(str).apply(lambda x : '-'.join((x[:4], x[4:6], x[6:8])) + ' '+ x[8:] + ':00:00'))
    prep_df.loc[prep_df['측정일시'].dt.hour == 0, '측정일시'] = prep_df.loc[prep_df['측정일시'].dt.hour == 0, '측정일시'] + datetime.timedelta(days = 1)
    
    del df
    gc.collect()
    
    prep_df = pd.pivot_table(prep_df, index = '측정일시', columns = '지역')
    prep_df = prepColumns(prep_df).reset_index()
    
    # merge를 위한 날짜 생성
    prep_df['측정일시'] = pd.to_datetime(prep_df['측정일시'])
    prep_df['연도'] = prep_df['측정일시'].dt.year
    prep_df['월'] = prep_df['측정일시'].dt.month
    prep_df['일'] = prep_df['측정일시'].dt.day
    prep_df['시간'] = prep_df['측정일시'].dt.hour
    prep_df = prep_df.drop('측정일시', axis = 1)
    return prep_df